# Лабораторная работа №3 — Деревья решений

Классификатор пола по голосу (датасет voice.csv).

Задания взяты из файла *Лабораторная работа #3 - Деревья решений - Классификатор пола по голосу.pdf*

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import tree, model_selection, metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Настройки графиков
sns.set(style="whitegrid", palette="muted")


In [ ]:

# Загрузка данных (предполагается, что voice.csv находится в папке data/)
voice_data = pd.read_csv("data/voice.csv")
voice_data.head()


In [ ]:

print("Размерность:", voice_data.shape)
print("\nПропуски:", voice_data.isnull().sum().sum())
voice_data.info()


In [ ]:

X = voice_data.drop("label", axis=1)
y = voice_data["label"]

# Train-test split (80/20)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


## Задание 1. Решающие пни (Decision Stumps)

- Построим дерево глубины 1 с критерием энтропии.
- Визуализируем.
- Ответим на вопросы.

In [ ]:

dt_stump = tree.DecisionTreeClassifier(max_depth=1, criterion="entropy", random_state=42)
dt_stump.fit(X_train, y_train)

plt.figure(figsize=(12, 6))
tree.plot_tree(dt_stump, feature_names=X.columns, class_names=dt_stump.classes_, filled=True)
plt.show()

# Фактор и порог в корне
root_feature = dt_stump.tree_.feature[0]
root_threshold = dt_stump.tree_.threshold[0]

print("Фактор в корне:", X.columns[root_feature])
print("Пороговое значение:", round(root_threshold, 3))

# Доля наблюдений в условии корня
mask = X_train.iloc[:, root_feature] <= root_threshold
print("Доля наблюдений в условии:", round(mask.mean() * 100, 1), "%")

# Accuracy
y_pred = dt_stump.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred), 3))


## Задание 2. Увеличим глубину дерева до 2

- Строим дерево глубины 2.
- Визуализируем.
- Проверяем, какие факторы используются.
- Считаем accuracy.

In [ ]:

dt_depth2 = tree.DecisionTreeClassifier(max_depth=2, criterion="entropy", random_state=42)
dt_depth2.fit(X_train, y_train)

plt.figure(figsize=(14, 7))
tree.plot_tree(dt_depth2, feature_names=X.columns, class_names=dt_depth2.classes_, filled=True)
plt.show()

# Факторы, которые использовались
features_used = set(X.columns[f] for f in dt_depth2.tree_.feature if f >= 0)
print("Факторы, использованные в дереве:", features_used)

# Количество листьев с классом female
leaf_values = dt_depth2.apply(X_train)
leaf_classes = dt_depth2.predict(X_train)
female_leaf_count = sum(
    (leaf_classes[leaf_values == node] == "female").any() 
    for node in np.unique(leaf_values)
)
print("Количество листьев с классом female:", female_leaf_count)

# Accuracy
y_pred2 = dt_depth2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred2), 3))


## Задание 3. Глубокое дерево без ограничения глубины

- Строим дерево с энтропией.
- Смотрим глубину, количество листьев.
- Сравниваем accuracy на train/test.

In [ ]:

dt_full = tree.DecisionTreeClassifier(criterion="entropy", random_state=0)
dt_full.fit(X_train, y_train)

print("Глубина дерева:", dt_full.get_depth())
print("Количество листьев:", dt_full.get_n_leaves())

y_train_pred = dt_full.predict(X_train)
y_test_pred = dt_full.predict(X_test)

print("Accuracy train:", round(metrics.accuracy_score(y_train, y_train_pred), 3))
print("Accuracy test:", round(metrics.accuracy_score(y_test, y_test_pred), 3))


## Задание 4. Подбор гиперпараметров (GridSearchCV)

- Используем критерии gini/entropy.
- max_depth от 4 до 10.
- min_samples_split [3, 4, 5, 10].

In [ ]:

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [3, 4, 5, 10]
}

cv = StratifiedKFold(n_splits=5)

grid = GridSearchCV(tree.DecisionTreeClassifier(random_state=0),
                    param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
print("Лучшие параметры:", grid.best_params_)

# Accuracy на train/test
y_train_best = best_model.predict(X_train)
y_test_best = best_model.predict(X_test)

print("Accuracy train:", round(metrics.accuracy_score(y_train, y_train_best), 3))
print("Accuracy test:", round(metrics.accuracy_score(y_test, y_test_best), 3))


## Задание 5. Важность факторов

- Визуализируем feature importances.
- Определим топ-3 факторов.

In [ ]:

importances = pd.Series(best_model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=importances_sorted.values, y=importances_sorted.index)
plt.title("Важность факторов (feature importance)")
plt.show()

print("Топ-3 признаков:")
print(importances_sorted.head(3))
